# Food Recommendation System

In [1]:
import sklearn
import numpy as np
import pandas as pd

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_columns', None)

In [2]:
temp = pd.read_csv('../input/foodcom-recipes-and-reviews/recipes.csv')
temp = temp[['Name', 'Description']]
temp['Name'] = temp['Name'].str.lower()
temp.head()

In [3]:
df = pd.read_csv('../input/ir-project/food_sample.csv')
df = df.merge(temp, how='left', on='Name')
df.head()

In [4]:
df['text'] = df['Name'].astype(str)+' '+df['Diet'].astype(str)+' '+df['Veg/Non-Veg'].astype(str)+' '+df['Ingredients'].astype(str)+' '+df['Disease'].astype(str)+' '+df['Description'].astype(str)
df.head()

## Preprocessing:

In [5]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_punctuation(data):
    symbols = """!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r"""
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_stop_words(data):
    words = word_tokenize(str(data))
    res = ' '.join([word for word in words if word not in cachedStopWords])
    return np.char.strip(res)

def tf(word, counter):
    return counter[word] / len(counter)

def tf_binary(word, counter):
    if counter[word]>0:
        return 1
    return 0

def tf_rawcount(word, counter):
    return counter[word]

def tf_lognorm(word, counter):
    return math.log(1+counter[word])

def idf(word, postings):
    return math.log(len(postings) / (1 + postings[word]))

def lemmatization(data):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
    return np.char.strip(new_text)

def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    data = lemmatization(data)
    return data

In [6]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [7]:
import re
import numpy as np
from tqdm import tqdm
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

cachedStopWords = stopwords.words("english")

In [8]:
def clean_text(data_df):
    for index, row in data_df.iterrows():
        sample = row['text']
        data_df.loc[index, 'text'] = str(preprocess(sample))
    return data_df

df = clean_text(df.copy())
df.head()

In [9]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800 , stopwords = STOPWORDS).generate(" ".join(df.text))
plt.imshow(wc , interpolation = 'bilinear')
plt.savefig('wordcloud.png')

## Model Stage 1:

In [10]:
!pip -q install sentence_transformers

In [11]:
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('bert-base-nli-mean-tokens')

In [12]:
sentence_embeddings = bert.encode(df['text']).tolist()

In [13]:
input_ = 'south indian'

processed_input = bert.encode([input_]).tolist()

In [14]:
df['Scores'] = cosine_similarity(processed_input, sentence_embeddings).reshape(-1)

filtered_df = df.sort_values('Scores', ascending=False).head(100).reset_index(drop=True)
filtered_df.head()

## Model Stage 2:

In [15]:
class Recommender:
    def __init__(self, data):
        self.df = data
        
        cat_dummies = self.df['Veg/Non-Veg'].str.get_dummies()
        nutrient_dummies = self.df['Nutrient'].str.get_dummies()
        disease_dummies = self.df['Disease'].str.get_dummies(sep=' ')
        diet_dummies = self.df['Diet'].str.get_dummies(sep=' ')
        feature_df = pd.concat([cat_dummies, nutrient_dummies, disease_dummies, diet_dummies], axis=1)
        
        self.data = feature_df
        
    def get_recommendations(self, inputs, k):
        total_features = self.data.columns
        temp = dict()
        for i in total_features:
            if i in inputs:
                temp[i] = 1
            else:
                temp[i] = 0
        
        temp = pd.DataFrame([temp])
        
        model = NearestNeighbors(n_neighbors=k, algorithm='ball_tree')
        model.fit(self.data)
        
        distances, indices = model.kneighbors(temp)
        
        results = self.df.iloc[indices[0]].reset_index(drop=True)
        results['Disance'] = distances[0]
        return results

## Inference:

In [16]:
ob = Recommender(filtered_df)
sample_input = ['veg', 'high_protien_diet', 'gluten_free_diet', 'diabeties', 'anemia', 'calcium', 'protien']

results = ob.get_recommendations(sample_input, 5)
results